# 使用朴素贝叶斯分类器从个人广告中获取区域倾向

1. 收集数据：从RSS源收集内容，这里需要对RSS源构建一个接口。
2. 准备数据：将文本文件解析成词条向量。
3. 分析数据：检查词条确保解析的正确性。
4. 训练算法：使用trainNB0()函数。
5. 测试算法：观察错误率，确保分类器可用。可以修改切分程序，以降低错误率，提高分类结果。
6. 使用算法：构建一个完整的程序，封装所有内容。给定两个RSS源，该程序会显示最常用的公共词。

## 收集数据：导入RSS源

In [1]:
import feedparser
import jieba
import random
import numpy as np
from math import log
import re

In [2]:
ny = feedparser.parse('http://www.caijing.com.cn/rss/stock_index.xml')

In [3]:
ny['entries']

[{'title': '昊志机电：控制人突击注销企业 治理结构堪忧',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'http://www.caijing.com.cn/rss/stock_index.xml',
   'value': '昊志机电：控制人突击注销企业 治理结构堪忧'},
  'summary': '此前，我们曾经从公司毛利率下滑、应收账款积累、产能利用率不足等角度分析了昊志机电存在的几大方面问题。这次，我们将继续深入解析该公司存在的几项风险因素，其一是公司实际控制人及其近亲属在申请上市前夕突...<br><a href="http://stock.caijing.com.cn/2014-08-25/114339054.html" target="_blank">[阅读全文]</a><br><a href="http://www.caijing.com.cn" target="_blank">[财经网]</a><br>',
  'summary_detail': {'type': 'text/html',
   'language': None,
   'base': 'http://www.caijing.com.cn/rss/stock_index.xml',
   'value': '此前，我们曾经从公司毛利率下滑、应收账款积累、产能利用率不足等角度分析了昊志机电存在的几大方面问题。这次，我们将继续深入解析该公司存在的几项风险因素，其一是公司实际控制人及其近亲属在申请上市前夕突...<br><a href="http://stock.caijing.com.cn/2014-08-25/114339054.html" target="_blank">[阅读全文]</a><br><a href="http://www.caijing.com.cn" target="_blank">[财经网]</a><br>'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'http://stock.caijing.com.cn/2014-08-25/114339054

In [4]:
len(ny['entries'])

50

## RSS源分类器及高频词去除函数

In [5]:
def calcMostFreq(vocabList, fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    sortedFreq = sorted(freqDict.items(), key=operator.itemgetter(1), reverse=True)
    return sortedFreq[:30]

In [6]:
# 对中文语句进行词分，采用搜索引擎模式分词
def textParse(bigString):
    import jieba
    # 正则：匹配汉字
    regEx = re.compile('.*?([\u4E00-\u9FA5]+)')
    bigStringList = regEx.split(bigString)
    listOfTokens = jieba.lcut_for_search(''.join(bigStringList))
    # 去除标点和单字
    return [tok for tok in listOfTokens if len(tok) > 1]

In [7]:
# 创建一个包含在所有文档中出现的不重复词的列表
def createVocabList(dataSet):
    # 创建一个空集合，为了去重
    vocabSet = set([])
    for document in dataSet:
        # 创建两个集合的并集
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

In [8]:
# 词袋模型
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [9]:
# trainMatrix-文档矩阵，trainCategory-由每篇文档类别标签所构成的向量
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    # 求出有侮辱性词语的文档占比
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    # 初始化概率，num为分子，denom为分母
    # 如果其中一个概率值为0， 那么最后的乘积也为0，为了降低这种影响，
    # 可以将所有词的出现数初始化为1， 并将分母初始化为2
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)
    p0Denom = 2.0
    p1Denom = 2.0
    for i in range(numTrainDocs):
        # p1表示侮辱性词语，p0表示正常词语
        if trainCategory[i] == 1:
            # 向量相加
            # 侮辱性词语的词频数向量
            p1Num += trainMatrix[i]
            # 侮辱性词语的总词数
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # 对每个元素做除法，求出该词的概率即p(w|c)
    # 解决下溢出问题：利用ln(a*b)=ln(a)+ln(b)
    p1Vect = np.array([log(x) for x in p1Num/p1Denom])
    p0Vect = np.array([log(x) for x in p0Num/p0Denom])
    return p0Vect, p1Vect, pAbusive

In [10]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    # 元素相乘，相当于p(w0|c1)p(w1|c1)...
    # 由于p(w)是一样的，所以在求p1的时候就不需要做除法运算了，计算p(c|w)
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

In [11]:
def localWords(feed1, feed0):
    docList=[]; classList=[]; fullText=[]
    minLen = min(len(feed1['entries']), len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    top30Words = calcMostFreq(vocabList, fullText)
    for pairW in top30Words:
        if pairW[0] in vocabList:
            vocabList.remove(pairW[0])
    trainingSet = range(2*minLen); testSet = []
    for i in range(20):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(list(trainingSet)[randIndex])
    trainMat = []; trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ', float(errorCount) / len(testSet))
    return vocabList, p0V, p1V

In [12]:
# 宏观经济RSS源
economy = feedparser.parse('http://www.caijing.com.cn/rss/economy_index.xml')

In [13]:
# 证券RSS源
stock = feedparser.parse('http://www.caijing.com.cn/rss/stock_index.xml')

In [14]:
vocabList, pEconnmy, pStock = localWords(economy, stock)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Hurf\AppData\Local\Temp\jieba.cache
Loading model cost 2.721 seconds.
Prefix dict has been built succesfully.


the error rate is:  0.05


## 分析数据：显示地域相关的用词

### 最具表征性的词汇显示函数

In [15]:
def getTopWords(economy, stock):
    vocabList, p0V, p1V = localWords(economy, stock)
    topEconomy = []; topStock = []
    for i in range(len(p0V)):
        # 设置-5.0的概率阈值
        if p0V[i] > -5.0:
            topEconomy.append((vocabList[i], p0V[i]))
        if p1V[i] > -5.0:
            topStock.append((vocabList[i], p1V[i]))
    sortedStock = sorted(topStock, key = lambda pair: pair[1], reverse=True)
    print('Stock**Stock**Stock**Stock**Stock**Stock**Stock**Stock**Stock**Stock')
    for item in sortedStock:
        print(item[0])
    sortedEconomy = sorted(topEconomy, key = lambda pair: pair[1], reverse=True)
    print('Economy**Economy**Economy**Economy**Economy**Economy**Economy**Economy**Economy**Economy')
    for item in sortedEconomy:
        print(item[0])

In [16]:
getTopWords(economy, stock)

the error rate is:  0.0
Stock**Stock**Stock**Stock**Stock**Stock**Stock**Stock**Stock**Stock
增长
全国
国人
会议
建设
中国人民银行
Economy**Economy**Economy**Economy**Economy**Economy**Economy**Economy**Economy**Economy
上市
股东
集团
发行
金额
成交
上市公司
代码
证券
股票名称
事项
收到
名称
存在
属于
股票代码
归属
归属于
重大
次序
公开
控股
发布
